# Introduction
In this document, we go step by step in the procedure of generating soil moisture prediction at any available resolution for a particular region. We illustrate this procedure estimating the soil moisture for a Km x Km centering the ignition point of the fire at The hike to Chimney Tops in the Great Smoky Mountains. 

Let us start introducing some terminology:

## Glossary
- USGS    : [The United States Geological Survey](https://www.usgs.gov/)
- SAGA    : It is an open-source software designed as a robust geospatioal anlysis package
- GIS     : Geographical Information System
- DEM     : Digital Elevation Model is a 3D CG representation of a terrain's surface
- CSV file: A CSV is a comma-separated values file, which allows data to be saved in a tabular format.
- Raster  : A raster is a file where is the data is stored as a matrix of cells also called pixeles. There exists a variaty of format of rasters, but in this document, we will work with three of them

    - **Rasters.img**: Usually, USGS save their raster in this format,
    - **Raster.tif** : It is the most used format for many applications
    - **Raster.sdat**: This is used by the package SAGA, which is required to the generate the terrestrail parameters

- Decimal Latitude and Longitude system: This a way to express geographic coordinates in decimal degrees. It’s just another way to represent that same location in a different format. For example, Chimney Tops is located at the coordinates (35.631167, -83.481167), which means Latitude =  35.631167  and  Longitude = -83.481167 
    

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%run tools.py

## We define some constants here. 
- **pi** is the value of the constant pi
- **Rearth** is the radius of earth
- **dx** is the distance from the center to any of the sides to the desired region
- **Year** corresponds to the year wwhen we want to estimate the soil moisture
- **Month** corresponds to the month of the year we want estimate the soil moisture

In [ ]:
# Define constants
pi = math.pi
REarth = 6378137 # Earth radius
dx     = 600     # Distance from  from the center

# Defining the folder where the file are stored
Path = %pwd
Folder = Path + '/Evaluation/'

In [ ]:
Folder


## Setting the name suffix to the files

In [ ]:
# Defining the dates to the predictions
Dates = [[2016,5],[2016,10]]

# The training file just dependens of the results
Training = 'Training'

# The Evaluation file just depends on the parameters and the region. It is constant through this file
Evaluating = 'Evaluation'

# The Moisture files depend on the Year, Month,and Models
Moisture = 'Moisture'


In [ ]:
Moisture

# 1. Selecting the Desired Region
Using any GIS software or web map viewer to find the desired region. In this example, we will work for a region containing the coordinates

In [ ]:
Lat  =  35.631167
Long = -83.481167


Which is the location where the fire started. Check in google earth to very the [location](https://earth.google.com/web/@35.63292863,-83.49585774,960.7674918a,19382.48366525d,35y,142.04295599h,45.11788942t,0r/data=CjkaNxIxCgAZP6iLFMrQQUAhhjqscMveVMAqGzM1wrAzNyc1Mi4yIk4gODPCsDI4JzUyLjIiVxgCIAE). Define the 1 Km square region bordering that point. In this point we need to add and substract 500 meters to the lattitude and longitude.

![Drag Racing](Images/ChimeneyTops.png)

In [ ]:
Max_Lat   = Lat  + (dx / REarth) * (180 / pi)
Min_Lat   = Lat  - (dx / REarth) * (180 / pi)
Max_Lon   = Long + (dx / REarth) * (180 / pi) / math.cos(Lat * pi/180)
Min_lon   = Long - (dx / REarth) * (180 / pi) / math.cos(Lat * pi/180)

print("Max Lat:", Max_Lat)
print("Min Lat:", Min_Lat)
print("Max Lon:", Max_Lon)
print("Min Lon:", Min_lon)

# 2. Select Digital Elevation Model (DEM) from USGS

Visit the webpage of The National Map viewer and download platform  https://viewer.nationalmap.gov/basic/
###  In the right panel, select coordinates
![Drag Racing](Images/ClickCoordinates.png)

### In the following windows, insert the coordinates and click on “Draw AOI”
![Drag Racing](Images/DrawnRegion.png)
![Drag Racing](Images/Chimney.png)

## In the left panel, select:
### Elevation Products (3DEP)
### The desired resolution, for this example, 1/9 arc-second DEM
###  Show availability
![Drag Racing](Images/SelectProducts.png)

### Continue selecting “find products” 
![Drag Racing](Images/Products.png)

**You do not need all of them, click on “Thumbnail”, and select just the products that cover your region of interest, and then click “view cart”. You will see that some tiles are spatially duplicated, as they are in areas overlaying two states, the same tiles are stored individually in different states’ topographic datasets. Select only one of the overlaying tiles.**
###  You can download the files one at the time, but it is recommended to Export URLs to text files and, then download them with a script

# 3. Download DEM files from USGS and Unzip them
Insert the name of the file where are the names of the tiles to be downloaded


In [ ]:
#Name of the file containing the files to be downloaded
filename = "DEMFiles.txt" 

# Download the files
download_dem(filename)    # Download the files

# Move the zip files to the folder 'Rasters'

!mv *.tif Rasters

# Unzip the zip files

os.chdir(r"Rasters")
#!unzip *.zip

os.chdir(r"../")

# Create a list of DEM Files
List_of_Rasters = !ls Rasters/*.tif


In [ ]:
List_of_Rasters

# 4. Merge all the DEM Rasters in a Unique Raster (Mosaicking)

We have seen that sometimes to cover a region we need many tiles. We need to merge all of these tiles in only one
![Drag Racing](Images/Mosaicking.png)

In [ ]:
mergin_raster(List_of_Rasters)
!mv Mosaicking.img Rasters/

# 5. Cropping the Desired Region
The Mosaic i.e., the merged raster covers more than the desired region, so we need to crop such region from the mosaic.
![Drag Racing](Images/Cropping.png)

In [ ]:
Max_lat   =  3946476
Min_lat   =  3945276 
Max_lon   =   276019
Min_lon   =   274819


croping_region('Rasters/Mosaicking.img', str(Min_lon), str(Max_lon), str(Min_lat), str(Max_lat), 'Rasters/Cropped.tif')

# 6. Change the Format of the Desired Region to SDAT format
The terrestrial parameters are generated using a package of SAGA, which requires all the rasters are in **SDAT format**


In [ ]:
File_name_of_region = 'Rasters/Cropped.tif'
convert_to_sdat(File_name_of_region)

# 7. Create the Terrestrial Parameters

The name of the file has to be input with **no extension**

In [ ]:
file = 'Rasters/Cropped'
terrestrial_parameters(file)

# 8. Create a Stack with the Elevation and all the Terrestrial Parameters
In this part is created four files:
- **A stack raster with all parameters in Latitude-Longitude format**
- ** A raster with all parameters in UTMformat **
- ** A CSV file with all parameters in Latitude-Longitude format **
- ** A CSV file with all parameters in UTM format **

They all are saved in the folder Rasters

In [ ]:
list_of_files = !ls Rasters/*.sdat
command = ['./CreatingStack.R'] + list_of_files
creating_stack(command)

# 9. Training the Models and Evaluating them in the Evaluating File
We need to import the training file to create the evaluation with the same parameters

In [ ]:
# Import the Training File
DataTime = {'Model': [], 'Time': []}
TiempoModelo = pd.DataFrame(DataTime)


Moisture_Files = []
for date in Dates:
    training = pd.read_csv(Folder+Training+'_'+str(date[0])+'_'+str(date[1])+'.csv')
    
    # Import the CSV File with all the parameters
    evaluation = pd.read_csv(Folder+'AllParametersLONGLAT.csv')
    
    # Determine what the parameters are used to train the model
    cols_training = list(training)

    # Extract the columns corresponding with the training file
    cols_Evaluation = cols_training[0:2]+cols_training[3:]

    # Rename the data frame with the corresponding columns
    evaluation = evaluation[cols_Evaluation]

    # Save the dataframe to a csv file in the Evaluation folder
    evaluation.to_csv(Folder+Evaluating+'_'+str(date[0])+'_'+str(date[1])+'.csv',index=False)

    # Training and Evaluating in the file
    for model in list(Models):
        training_file    = Folder+Training+'_'+str(date[0])+'_'+str(date[1])+'.csv'
        evaluation_file  = Folder+Evaluating+'_'+str(date[0])+'_'+str(date[1])+'.csv'
        moisture_file    = Folder+Moisture+'_'+str(model)+'_'+str(date[0])+'_'+str(date[1])+'.csv'
        start_time = time.time()
        creating_moisture(model, training_file, evaluation_file, moisture_file)
        final_time = time.time()-start_time
        TiempoModelo=TiempoModelo.append({'Model':str(model)+'_'+str(date[0])+'_'+str(date[1]), 'Time':final_time}, ignore_index=True)
        
        Moisture_Files = Moisture_Files + [moisture_file]
    
TiempoModelo.to_csv(Folder+'TiempoModelo.csv',index=False)    

In [ ]:
TiempoModelo['Model']

# 11. Saving  the  Moisture and the Elevation Dataframe in a Unique File

In [ ]:
Columna_evaluacion = list(evaluation['CONUSDEM1km'])

In [ ]:
LatLongFiles = []
for file in Moisture_Files:
    print(file)
    # Load the Soil Moisture File
    soil_moisture2 = pd.read_csv(file,usecols=[0,1,2], names = ['x','y','Moisture'])   

    # Add a new column with the elevation
    soil_moisture = evaluation.copy()
    soil_moisture['Moisture']= soil_moisture2['Moisture']
    soil_moisture = soil_moisture[['x','y','CONUSDEM1km','Moisture']] 
    soil_moisture.rename(columns={"CONUSDEM1km": "Elevation"})
    #soil_moisture['Elevation'] = Columna_evaluacion
    

    # Save the soil_moisture to a CSV file
    soil_moisture.to_csv(file[:-4]+'_latlong.csv',index=False)
    LatLongFiles = LatLongFiles + [file[:-4]]

# 12. Change the Coordinates to UTM
All the files are expressed in Latitude-Longitude format. We need to change soil moisture and elevation file to UTM format (meters)

In [ ]:
UTM_Files = []
for file in LatLongFiles:
    Input_file  =  file+'_latlong.csv'
    Output_file =  file+'_UTM.csv'
    UTM_Files = UTM_Files + [Output_file]
    
    # Changing the format
    changing_to_utm(Input_file, Output_file)

# 12. Moving the region such that the left-botton corner be located at (0,0)



In [ ]:
for utm_file in UTM_Files:
    # Load the file
    EvaluationUTM = pd.read_csv(utm_file)
    

    # Round the coordinates to have integer numbers
    EvaluationUTM['x'] = EvaluationUTM['x'].round()
    EvaluationUTM['y'] = EvaluationUTM['y'].round()
    Traslate_x = EvaluationUTM['x'].min()+100
    Traslate_y = EvaluationUTM['y'].min()+100
    
    # Shift the coordinates to (0,0)
    EvaluationUTM['x'] = EvaluationUTM['x']- Traslate_x
    EvaluationUTM['y'] = EvaluationUTM['y']- Traslate_y

    # Sort the values
    EvaluationUTM = EvaluationUTM.sort_values(by=['x', 'y'])


    EvaluationUTM = EvaluationUTM[EvaluationUTM['x'] >-1]
    EvaluationUTM = EvaluationUTM[EvaluationUTM['y'] >-1]
    EvaluationUTM = EvaluationUTM[EvaluationUTM['x'] <1000]
    EvaluationUTM = EvaluationUTM[EvaluationUTM['y'] <1000]
    
    #Drop duplicated coordinates in case there are some
    EvaluationUTM = EvaluationUTM.drop_duplicates(subset=['x', 'y'], keep='first')
    
    EvaluationUTMGIS = EvaluationUTM.copy()
    EvaluationUTMGIS['x'] = EvaluationUTMGIS['x'] + Traslate_x
    EvaluationUTMGIS['y'] = EvaluationUTMGIS['y'] + Traslate_y


    # Save the dataframe to a CSV file
    EvaluationUTMGIS.to_csv(utm_file, index=False)
    EvaluationUTM.to_csv(utm_file[:-4]+'_Traslated.csv', index=False)

# 13. Plotting the Soil Moisture for The desired Region 
soil_map(df, title="Soil Moisture Heatmap", out="", cmap=cmap_rg, legend="Soil Moisture", size=.05, vmin=None, vmax=None, value=2):

In [ ]:
Modelos = ['Moisture_KKNN_2016_5_latlong.csv','Moisture_KKNN_2016_10_latlong.csv',\
          'Moisture_SBM_2016_5_latlong.csv','Moisture_SBM_2016_10_latlong.csv',\
          'Moisture_HYPPO_2016_5_latlong.csv','Moisture_HYPPO_2016_10_latlong.csv',\
          'Moisture_RF_2016_5_latlong.csv','Moisture_RF_2016_10_latlong.csv']


In [ ]:
minimo = 0
maximo =1;
for model in Modelos:
    df =  pd.read_csv('Evaluation/'+model)
    df = df[['x','y','Moisture']]
    if df['Moisture'].min() > minimo:
        minimo = df['Moisture'].min()
    if df['Moisture'].max() < maximo:
        maximo = df['Moisture'].max()
        


In [ ]:
for model in Modelos:
    df =  pd.read_csv('Evaluation/'+model)
    df = df[['x','y','Moisture']]
    soil_map(df, out='Out_Images/'+model[0:-4]+'.png', title=model[0:-12],cmap=plt.cm.get_cmap('RdBu'),vmin=minimo,vmax=maximo)